# Amazon Price Tracker Scraper (Configurable)
A ready‑to‑run scraper that logs daily price data for **any** Amazon product. Just paste the product URL in the configuration cell.

In [1]:
# 1. IMPORT LIBRARIES
import requests
from bs4 import BeautifulSoup
import csv, datetime as dt, time, os, pandas as pd
import matplotlib.pyplot as plt
plt.style.use('default')

In [3]:
# 2. CONFIGURATION 
# 🔗 Paste **any** Amazon product URL below.
URL = "https://www.amazon.com/dp/B095KS3FBG/ref=sbl_dpx_kitchen-electric-cookware_B095KS3FBG_00?th=1"  

HEADERS = {
    "User-Agent": (
        "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) "
        "AppleWebKit/537.36 (KHTML, like Gecko) "
        "Chrome/123.0.0.0 Safari/537.36"
    ),
    "Accept-Language": "en-US,en;q=0.9",
}

CSV_FILE = "amazon_price_history.csv"   
CHECK_INTERVAL_HRS = 24                  
TARGET_PRICE = 59.99                     

In [5]:
# 3. SCRAPER FUNCTION 
def scrape_amazon(url: str):
    """Return (title, price) for the product."""
    resp = requests.get(url, headers=HEADERS, timeout=30)
    resp.raise_for_status()
    soup = BeautifulSoup(resp.text, "lxml")
    title_tag = soup.find(id="productTitle")
    title = title_tag.get_text(strip=True) if title_tag else "Unknown product"
    price_tag = soup.find("span", class_="a-offscreen")
    price = None
    if price_tag:
        try:
            price = float(price_tag.get_text(strip=True).replace("$", ""))
        except ValueError:
            pass
    return title, price

In [7]:
# 4. DATA STORAGE
def log_price(title: str, price):
    today = dt.datetime.now().date()
    file_exists = os.path.isfile(CSV_FILE)
    with open(CSV_FILE, "a", newline="", encoding="utf-8") as f:
        writer = csv.writer(f)
        if not file_exists:
            writer.writerow(["date", "title", "price"])
        writer.writerow([today, title, price])

In [9]:
# 5. OPTIONAL E‑MAIL ALERT 
# Uncomment + configure to receive alerts when price < TARGET_PRICE.
'''
import smtplib, ssl
from email.message import EmailMessage
GMAIL_USER = os.getenv("GMAIL_USER")
GMAIL_PASS = os.getenv("GMAIL_APP_PASS")
def send_email(title, price):
    msg = EmailMessage()
    msg["Subject"] = f"Price drop: {title} now ${price:.2f}"
    msg["From"] = GMAIL_USER
    msg["To"] = GMAIL_USER
    msg.set_content(f"Buy now: {URL}")
    ctx = ssl.create_default_context()
    with smtplib.SMTP_SSL("smtp.gmail.com", 465, context=ctx) as server:
        server.login(GMAIL_USER, GMAIL_PASS)
        server.send_message(msg)
'''

'\nimport smtplib, ssl\nfrom email.message import EmailMessage\nGMAIL_USER = os.getenv("GMAIL_USER")\nGMAIL_PASS = os.getenv("GMAIL_APP_PASS")\ndef send_email(title, price):\n    msg = EmailMessage()\n    msg["Subject"] = f"Price drop: {title} now ${price:.2f}"\n    msg["From"] = GMAIL_USER\n    msg["To"] = GMAIL_USER\n    msg.set_content(f"Buy now: {URL}")\n    ctx = ssl.create_default_context()\n    with smtplib.SMTP_SSL("smtp.gmail.com", 465, context=ctx) as server:\n        server.login(GMAIL_USER, GMAIL_PASS)\n        server.send_message(msg)\n'

In [ ]:
# 6. MAIN LOOP 
def main():
    while True:
        title, price = scrape_amazon(URL)
        log_price(title, price)
        print(f"{dt.datetime.now()}: {title[:60]} … ${price}")
        # if price is not None and price < TARGET_PRICE:
        #     send_email(title, price)
        time.sleep(CHECK_INTERVAL_HRS * 3600)

# Call main() only when running the notebook as a script.
if __name__ == "__main__":
    main()

2025-05-04 22:11:38.494761: Ninja Foodi Programmable 10-in-1 5-Quart Pressure Cooker and … $128.95


## Quick Exploratory Analysis
After running for a few days, visualise the collected price history:

In [ ]:
df = pd.read_csv(CSV_FILE, parse_dates=["date"])
df.tail()

In [ ]:
plt.figure(figsize=(8,4))
plt.plot(df["date"], df["price"], marker="o")
plt.title("Amazon Price History")
plt.xlabel("Date")
plt.ylabel("Price (USD)")
plt.grid(True)
plt.tight_layout()
plt.show()